## Tracking
#### Kalman filter (continuous) and (uni-modal)

#### Monte-carlo localization (discrete) and (multi-modal)

#### Particle filters (continuous) and (multi-modal)

Kalman filter approximates gaussian distribution (a continuous function over the space of locations and the area underneath sums to 1.

Gaussian characterised by two parameters, mean(mu) and variance(sigma2).

    1-D Gaussian (mu, sigma2)
    
    const = 1/(sqrt(2. * Pi * Sigma2)
            
    f(x) = const * exp^(-0.5 * ((x - mu)^2)/sigma2))

#### Calculate f(x) for:
    mu = 10
    sigma2 = 4
    x = 8
    (x-mu)^2 = 4
    (x-mu)^2/sigma2 = 4/4 = 1
    

In [123]:
import numpy as np

mu = 10
sigma2 = 4
x = 8
const = 1/(np.sqrt(2*np.pi*sigma2))
term2 = np.exp(-0.5 * (x - mu)**2/sigma2)
fx = const * term2
fx

0.12098536225957168

Create gaussian function

    def f(mu, sigma2, x):
        return 1/sqrt(2.*pi*sigma2) * exp(-.5 * (x-mu)**2 / sigma2)

In [124]:
from math import *

def f(mu, sigma2, x):
    return 1/sqrt(2.*pi*sigma2) * exp(-.5*(x-mu)**2 / sigma2)

print(f(10.,4.,8.))

0.12098536225957168


The peak of the gaussian is where x = mu

In [125]:
print(f(10., 4., 10.))

0.19947114020071635


## Localization

BELIEF = PROBABILITY

MEASUREMENT = PRODUCT FOLLOWED BY NORMALIZATION

MOVE = CONVOLUTION (=ADDITION)

### Measurements  =  Bayes Rule (product)


### Motion(prediction)  =  Total Probability (convolution)


    Measurement means updating our belief and renormalizing our distribution.
    
    Motion means keeping track of changes in probability using the law of Total Probability.

Gain information from 2 component gaussians, yielding a peakier posterior distribution after multiplying according to Bayes rule (higher median).

Fusing two component gaussians yields a mean somewhere in the middle of the two distributions.

Common situation for measurement update step:

    prior p(x)
    measurement probability p(z|x)
    posterior p(x|z)
    
Wide low prior p(x), updated through Bayes rule product with measurement update p(z|x) yields higher median, lower variance posterior distribution p(x|z), with mean between the two original distributions.

### Parameter Update

    Two distributions with (mean, variance):
        p(x) = mu, sigma2
        p(z|x) = nu, r2
        p(x|z) = ???
        
    After update:
    
        muP = (r2 * mu + sigma2 * nu) / (r2 + sigma2)
        sigma2P = 1 / (1/r2 + 1/sigma2)
        

In [126]:
def mu_prime(mu, sigma2, nu, r2):
    return (r2 * mu + sigma2 * nu) / (r2 + sigma2)
    
def sigma_prime(sigma2, r2):
    return 1 / (1/r2 + 1/sigma2)

In [127]:
mu = 10
sigma2 = 4
nu = 12
r2 = 4

muP = mu_prime(mu, sigma2, nu, r2)
sigma2P = sigma_prime(sigma2, r2)
print("Mean prime: ", muP)
print("Variance prime: ", sigma2P)

Mean prime:  11.0
Variance prime:  2.0


In [128]:
mu = 10
sigma2 = 8
nu = 13
r2 = 2

muP = mu_prime(mu, sigma2, nu, r2)
sigma2P = sigma_prime(sigma2, r2)
print("Mean prime: ", muP)
print("Variance prime: ", sigma2P)

Mean prime:  12.4
Variance prime:  1.6


### Update function is combination of the above two functions

     mu = mean1 and nu = mean2

     sigma2 = var1 and r2 = var2
     

    def update(mean1, var1, mean2, var2):
    
        new mean = (var2 * mean1 + var1 * mean2) / (var2 + var1)
        
        new_var = 1 / (1/var2 + 1/var1)

In [129]:
def update(mean1, var1, mean2, var2):
    new_mean = (var2 * mean1 + var1 * mean2) / (var2 + var1)
    new_var = 1 / (1/var2 + 1/var1)
    return [new_mean, new_var]

In [130]:
print(update(10.,8.,13., 2.))

[12.4, 1.6]


### Movement (convolution) is an addition

    mu, sigma2 = 8, 4
    
    nu, r2 = 10, 6

In [131]:
mean1, var1 = 8, 4  
mean2, var2 = 10, 6
muP = mean1+mean2
sigma2P = var1+var2

In [132]:
print("Mean after movement: ", muP)
print("Variance after movement: ", sigma2P)

Mean after movement:  18
Variance after movement:  10


In [133]:
def predict(mean1, var1, mean2, var2):
    new_mean = mean1+mean2
    new_var = var1+var2
    return [new_mean, new_var]

In [134]:
muP, sigma2P = predict(mean1, var1, mean2, var2)
print("Mean after movement: ", muP)
print("Variance after movement: ", sigma2P)

Mean after movement:  18
Variance after movement:  10


### Update function: used to update mean/variance based on measurements

### Predict function is used to predict new mean/variance after movement.

In [135]:
measurements = [5., 6., 7., 9., 10.]
motion = [1., 1., 2., 1., 1.]
measurement_sig = 4.
motion_sig = 2.
mu = 0.
sig = 1000.

In [136]:
for i in range(len(measurements)):
    [mu, sig] = update(mu, sig, measurements[i], measurement_sig)
    print('Mean/var after measurement: ',[mu, sig])
    [mu, sig] = predict(mu, sig, motion[i], motion_sig)
    print('Mean/var after motion:      ', [mu, sig])

Mean/var after measurement:  [4.9800796812749, 3.9840637450199203]
Mean/var after motion:       [5.9800796812749, 5.98406374501992]
Mean/var after measurement:  [5.992019154030327, 2.3974461292897047]
Mean/var after motion:       [6.992019154030327, 4.397446129289705]
Mean/var after measurement:  [6.996198441360958, 2.094658810112146]
Mean/var after motion:       [8.996198441360958, 4.094658810112146]
Mean/var after measurement:  [8.99812144836331, 2.0233879678767672]
Mean/var after motion:       [9.99812144836331, 4.023387967876767]
Mean/var after measurement:  [9.99906346214631, 2.0058299481392163]
Mean/var after motion:       [10.99906346214631, 4.005829948139216]


### Multivariate Gaussians (High-dimensional)

    Mean now a vector with one element for each of the dimensions, D and variance term replaced by covariance matrix with D rows and D columns if dimensions of estimate is D:
    
    mu = [mu0, mu1, ...muD]
    
    covar = np.array[[..., ...,
                     ..., ...,
                     ..., ...,]]

## 2D Kalman Filters

Prior gaussian (result of predict function/movement) multiplied by measurement probability(result of update function).

Resulting gaussian yields good estimate of velocity and location.

    xprime (location after timestep) = x (old location) + xdot (velocity)
    
        ** Remember this assumes time interval delta_t to be 1. A more general formula would be xprime = x + xdot * delta_t
    
Kalman filter variables (states) allow us through multiple observations (such as of momentary location) to infer hidden variables (such as velocity).

### Prediction step

    x = estimate
    F = state transition matrix
    u = motion vector
    P = uncertainty covariance
    
    xprime = F * x + u
    Pprime = F * P * F.T
    
### Measurement update step

    z = measurement
    H = measurement function
    y = error
    R = measurement noise
    I = identity matrix
    
    y = z - H * x
    S = H * P * H.T + R
    K = P * H.T * (1/S)
    xprime = x + (K * y)
    Pprime = (I - K * H) * P
    
    

### Matrix class for matrix manipulation

In [137]:
import matrix

In [138]:
a = matrix.matrix([[10.], [10.]])
a.show()

[10.0]
[10.0]
 


In [139]:
b = a.transpose()
b.show()

[10.0, 10.0]
 


In [140]:
F = matrix.matrix([[12., 8.], [6., 2.]])
b = F * a
b.show()

[200.0]
[80.0]
 


### Kalman filter design

### Prediction step
  
    xprime = F * x + u
    Pprime = F * P * F.T
    
### Measurement update step

    y = z - H * x
    S = H * P * H.T + R
    K = P * H.T * (1/S)
    
    xprime = x + (K * y)
    Pprime = (I - K * H) * P
    
    x = intial state (location and velocity)
    P = initial uncertainty
    u = external motion
    F = next state function
    H = measurement function
    R = measurement uncertainty
    I = identity matrix
    Z = measurement

In [141]:
measurements = [1, 2, 3]

x = matrix.matrix([[0.], [0.]])
P = matrix.matrix([[1000., 0.], [0., 1000.]])
u = matrix.matrix([[0.], [0.]])
F = matrix.matrix([[1., 1.], [0, 1.]])
H = matrix.matrix([[1., 0.]])
R = matrix.matrix([[1.]])
I = matrix.matrix([[1., 0.], [0., 1.]])

In [142]:
def kalman_filter(x, P):
    for i in range(len(measurements)):
        Z = matrix.matrix([[measurements[i]]])
        y = Z - (H * x)
        S = H * P * H.transpose() + R
        K = P * H.transpose() * S.inverse()
        
        # measurement update step
        x = x + (K * y)
        P = (I - (K * H)) * P
        
        # prediction (move) step
        x = (F * x) + u
        P = F * P * F.transpose()
        
    return x,P

In [143]:
print((kalman_filter(x, P)))

([[3.9996664447958645], [0.9999998335552873]], [[2.3318904241194827, 0.9991676099921091], [0.9991676099921067, 0.49950058263974184]])


### 4D Kalman Filters (hard)

    measurements = sequence of measurements in 2D space (x, y)
    
    initial_xy = initial (x, y) location
    
                    [4., 12.]
    
    dt = 0.1 (tenth of a timestep)
    
    x = initial state vector set up: (initial x, initial y, 0, 0) 
        "0" for unknown velocities
        
          [[initial_xy[0]], [initial_xy[1]], [0.], [0.]]
        
    u = all zeros because no "external" or outside force involved
    
                   [[0.], [0.], [0.], [0.]]
    
    P = initial uncertainty: 0 for positions x and y, 1000 for the two velocities (**along main diagonal of the matrix)
    
                    [[0., 0., 0., 0],
                     [0., 0., 0., 0.],
                     [0., 0., 1000., 0.],
                     [0., 0., 0., 1000.]]
    
    F = next state function all 1s along the main diagonal because position and velocity expected to remain the same, dt in off diagonals because time interval and velocity together impact both x and y coordinates
    
    
    H = 4x2 projection matrix projecting beliefs about x and y without velocities (ie 2 row matrix, with 1 in first entry first row for x coordinate and 1 in second row second entry for y:
    
                    [[1., 0., 0., 0.],
                     [0., 1., 0., 0.]]
                    
    R =  measurement uncertainty: use 2x2 matrix with 0.1 as main diagonal
    
                        [[0.1, 0],
                         [0, 0.1]]
                         
    I = 4D identity matrix (ie matrix with 4 rows, 4 columns and 1s across main diagonal.
    
                    [[1., 0., 0., 0.],
                     [0., 1., 0., 0.],
                     [0., 0., 1., 0.],
                     [0., 0., 0., 1.]]

In [144]:
measurements = [[5., 10.], [6., 8.], [7., 6.], [8., 4.], [9., 2.], [10., 0.]]
initial_xy = [4., 12.]

dt = 0.1

x = matrix.matrix([[initial_xy[0]], [initial_xy[1]], [0.], [0.]])
u = matrix.matrix([[0.], [0.], [0.], [0.]])

In [145]:
P =  matrix.matrix([[0., 0., 0., 0],
                    [0., 0., 0., 0.],
                    [0., 0., 1000., 0.],
                    [0., 0., 0., 1000.]])

F =  matrix.matrix([[1., 0., dt, 0.],
                    [0., 1., 0., dt],
                    [0., 0., 1., 0.],
                    [0., 0., 0., 1.]])
H =  matrix.matrix([[1., 0., 0., 0.],
                    [0., 1., 0., 0.]])

R =  matrix.matrix([[0.1, 0.],
                    [0., 0.1]])

I =  matrix.matrix([[1., 0., 0., 0.],
                    [0., 1., 0., 0.],
                    [0., 0., 1., 0.],
                    [0., 0., 0., 1.]])

In [146]:
def kalman_filter_4d(x, P):
    for n in range(len(measurements)):
        
        # prediction
        x = (F * x) + u
        P = F * P * F.transpose()
        
        # measurement update
        Z = matrix.matrix([measurements[n]])
        y = Z.transpose() - (H * x)
        S = H * P * H.transpose() + R
        K = P * H.transpose() * S.inverse()
        x = x + (K * y)
        P = (I - (K * H)) * P
        
    return x, P

In [147]:
xprime, Pprime = (kalman_filter_4d(x, P))

In [148]:
print("x = \n", xprime)
print(" ")
print("P = \n", Pprime)

x = 
 [[9.999340731787717], [0.001318536424568617], [9.998901219646193], [-19.997802439292386]]
 
P = 
 [[0.03955609273706198, 0.0, 0.06592682122843721, 0.0], [0.0, 0.03955609273706198, 0.0, 0.06592682122843721], [0.06592682122843718, 0.0, 0.10987803538073201, 0.0], [0.0, 0.06592682122843718, 0.0, 0.10987803538073201]]


### Other examples

In [149]:
measurements = [[1., 4.], [6., 0.], [11., -4.], [16., -8.]]
initial_xy = [-4., 8.]

x = matrix.matrix([[initial_xy[0]], [initial_xy[1]], [0.], [0.]])

xprime, Pprime = (kalman_filter_4d(x, P))

print("x = \n", xprime)
print(" ")
print("P = \n", Pprime)

x = 
 [[15.993335554815062], [-7.99466844385205], [49.98333888703765], [-39.98667110963012]]
 
P = 
 [[0.05331556147950691, 0.0, 0.13328890369876803, 0.0], [0.0, 0.05331556147950691, 0.0, 0.13328890369876803], [0.1332889036987679, 0.0, 0.33322225924692717, 0.0], [0.0, 0.1332889036987679, 0.0, 0.33322225924692717]]


In [150]:
measurements = [[1., 17.], [1., 15.], [1., 13.], [1., 11.]]
initial_xy = [1., 19.]

x = matrix.matrix([[initial_xy[0]], [initial_xy[1]], [0.], [0.]])

xprime, Pprime = (kalman_filter_4d(x, P))

print("x = \n", xprime)
print(" ")
print("P = \n", Pprime)

x = 
 [[1.0], [11.002665778073975], [0.0], [-19.993335554815054]]
 
P = 
 [[0.05331556147950691, 0.0, 0.13328890369876803, 0.0], [0.0, 0.05331556147950691, 0.0, 0.13328890369876803], [0.1332889036987679, 0.0, 0.33322225924692717, 0.0], [0.0, 0.1332889036987679, 0.0, 0.33322225924692717]]
